### Task 1 - Binary Representations
<hr>

Rotating bits refers to shifting bits in a direction, but instead of dropping the shifted bits, appending them to the other end of the integer.
For example, 01010101 rotated by 1 to the right will become 10101010.

This is done in **rotl** using bitwise operations like >> and |.
It iterates over the last n few bits in the binary representation of the number, and adds all of them to a string.
This string is then converted back into an integer, and appended to the start of the bits.
- This is done by shifting the bits to append by the number of bits in the original number. So for example if the bits to append are 110, and the number of bits in the original number 10, 10 zeroes are added to the end of appendBits.
- Then, this is OR'd against the original number right-shifted by n, the last few bits removed to be added on.
- This creates a binary number with the rotated bits at the start, followed by the original bits minus the bits rotated from the end.

**rotr** is similar

In [57]:
def rotl(x, n): # Rotate the bits to the left by n
    print(f"Before Rotation: 0b{x:032b}")
    s = "0b" # Keeps track of the rotated bits to be appended to the start.


    for i in range(n):
        rotatedBit = (x >> i) & 1 # Bit at the rightmost position of the number currently.
        s = s + (str)(rotatedBit) # Add that bit to the strng.

    appendBits = int(s, 2)

    # len(bin(x))-n+2 gives the length of the binary number without the prefix or the rotated bits.
    result = (appendBits << (len(bin(x))-(n+2))) | (x >> n) # [1], appending bits to the start of a binary number.

    return result

y = rotl(345436534, 3)
padding = f"After Rotation: 0b{y:032b}"
print(padding)

Before Rotation: 0b00010100100101101111000101110110
After Rotation: 0b00001110100100101101111000101110


In [86]:
def rotr(x, n): # Rotate the bits to the right by n
    print(f"Before Rotation: 0b{x:0b}")

    s = "0b" # Keeps track of the rotated bits to be appended to the end.

    for i in range(n):
        rotatedBit = (x >> (len(bin(x))-i)-3) & 1
        s = s + (str)(rotatedBit)
        print(rotatedBit)

    appendBits = (int)(s,2)

    result = ((x << n) | appendBits) >> n

    print(bin(result))


rotr(23235, 7)

Before Rotation: 0b101101011000011
1
0
1
1
0
1
0
0b101101011000011


In [ ]:
def ch(x,y,z): # Choose the bits from y where x has bits set to 1, and bits in z where x has bits set to 0
    print() # & operator should be used here.

In [ ]:
def maj(x,y,z) # Takes a majority vote of the bits in x,y,z. Output should have 1 in the position where at least two of x,y,z have 1.
    print()

References
- [1] Appending bits to the start of a binary number: https://stackoverflow.com/a/51678298

